The Data has been uploaded to our google drive
lets now added to the collab directory to ensure fast loading.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cp /content/drive/My\ Drive/vgg19/kvasir-dataset-v2.zip .

In [ ]:
! 7z x kvasir-dataset-v2.zip -aoa -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2489312085 bytes (2374 MiB)

Extracting archive: kvasir-dataset-v2.zip
 51% 4096 Open              --
Path = kvasir-dataset-v2.zip
Type = zip
Physical Size = 2489312085

  0%      0% 50 - kvasir-dataset-v2/polyps/b5af7a66-7f29-4942-89d2-d6a02126c123.jpg                                                                             0% 94 - kvasir-dataset-v2/polyps/1220f40b-25aa-4d32-9980-29b59f92ae7f.jpg                                                             

In [ ]:
import os
import numpy as np
import seaborn as sea
import tensorflow as tf 
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow.keras.applications as Applications
import tensorflow.keras.preprocessing.image as ImagePreprocessing
import cv2
import pickle
import json
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense ,MaxPool2D ,BatchNormalization,Flatten 
from sklearn.model_selection import KFold
from tensorflow.keras.applications.resnet50 import ResNet50
from PIL import Image
import importlib.util
import random as rn
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copyfile
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json

sns.set()

In [ ]:
train_split = 0.7
val_split = 0.15
test_split = 0.15
original_ds_path = 'kvasir-dataset-v2'
train_ds_path = 'train'
val_ds_path = 'validate'
test_ds_path = 'test'
base_dir = '/content/drive/MyDrive/vgg19'
losses_dir = base_dir+'/losses/'
model_save= base_dir+'/model_saves/'
seed = 49
size_img = (224,224) #size choosen to preserve the aspect ratio
input_size = (224,224,3)
batch_size = 32

Now let's split the data into train, validate test 
for this lets split randomly as 70% 15% and 15% respectively

In [ ]:
ds_dic = {}
for folder in os.listdir(original_ds_path):
  ds_dic[folder] = os.listdir(original_ds_path+'/'+folder) 

In [ ]:
if not os.path.exists(train_ds_path):
      os.mkdir(train_ds_path)
if not os.path.exists(val_ds_path):
      os.mkdir(val_ds_path)
if not os.path.exists(test_ds_path):
      os.mkdir(test_ds_path)

for each in ds_dic:
    
    if not os.path.exists(train_ds_path+'/'+each+'/'):
        os.mkdir(train_ds_path+'/'+each+'/')
    if not os.path.exists(val_ds_path+'/'+each+'/'):
        os.mkdir(val_ds_path+'/'+each+'/')
    if not os.path.exists(test_ds_path+'/'+each+'/'):
        os.mkdir(test_ds_path+'/'+each+'/')

In [ ]:
for each in ds_dic.keys():
  train_split , intermediate_split=train_test_split(ds_dic[each], 
                                              test_size=0.3, 
                                              random_state=seed,
                                              shuffle = True)
  
  val_split , test_split = train_test_split(intermediate_split, 
                                              test_size=0.5, 
                                              random_state=seed,
                                              shuffle = True)
  

  #print(len(train_split))
  #print(len(test_split))
  #print(len(val_split))

  for img in train_split:
        destination = train_ds_path+'/'+each+'/'+img
        source = original_ds_path+'/'+each+'/'+img
        
        if not os.path.exists(destination):
            copyfile(source, destination)
        
  for img in test_split:
      destination = test_ds_path+'/'+each+'/'+img
      source = original_ds_path+'/'+each+'/'+img
      
      if not os.path.exists(destination):
          copyfile(source, destination)

  for img in val_split:
      destination = val_ds_path+'/'+each+'/'+img
      source = original_ds_path+'/'+each+'/'+img
      
      if not os.path.exists(destination):
          copyfile(source, destination)


In [ ]:
train_datagen = ImageDataGenerator(featurewise_center=True,
                               samplewise_center=True,
                               featurewise_std_normalization=True,
                               samplewise_std_normalization=True,
                               zca_whitening=False,
                               zca_epsilon=1e-06,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2, 
                               brightness_range=None, 
                               shear_range=0.2, 
                               zoom_range=0.2,
                               channel_shift_range=0.0, 
                               fill_mode='nearest', 
                               cval=0.0, 
                               horizontal_flip=True,
                               vertical_flip=True, 
                               rescale=1./255, 
                               preprocessing_function=None,
                               validation_split=0.15
    )

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(directory=train_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )
val_generator = val_datagen.flow_from_directory(directory=val_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )
test_generator = test_datagen.flow_from_directory(directory=test_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )

Found 5600 images belonging to 8 classes.
Found 1200 images belonging to 8 classes.
Found 1200 images belonging to 8 classes.


In [ ]:
Base_Model = tf.keras.applications.VGG19(input_tensor=tf.keras.Input(shape=input_size),
                      weights='imagenet',
                      include_top=False)

Base_Model.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [ ]:
for layer in Base_Model.layers:
    layer.trainable = False

In [ ]:
conn_layer = Base_Model.get_layer('block5_pool')
conn_output = conn_layer.output
x = conn_output 
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Flatten()(x)
#x = Conv2D(512,(3,3),activation='relu')(conn_output)
#x = BatchNormalization()(x)
#x = Conv2D(256,(3,3),activation='relu')(x)
x = Dense(512,activation='relu')(x)
x = Dense(256,activation='relu')(x)
x = Dense(128,activation='relu')(x)
predictions = Dense(8,activation='softmax')(x)

In [ ]:
model = tf.keras.Model(inputs=Base_Model.input, outputs=predictions)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adamax(lr=1e-2), 
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                       tf.keras.metrics.TruePositives(name='true_positive'),
                       tf.keras.metrics.TrueNegatives(name='true_negatives')
                       ]
              )

In [ ]:
try:
  if not os.path.exists(base_dir+'model_saves'):
    os.mkdir(model_save)
except FileExistsError:
  print('FileExists')

FileExists


In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:
i = 0
for layer in model.layers: 
  print(i,layer)
  i+=1

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7ff4ece5ae10>
1 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff49d71db10>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff49ce06e10>
3 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7ff49ce06b10>
4 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff49c919e50>
5 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff49c923810>
6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7ff49cd96f90>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff49c92cf50>
8 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff49c930f90>
9 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff49c8b5510>
10 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff49c8bad90>
11 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7ff49c8bfe10>
12 <tensorflow.python.ke

In [ ]:
for layer in model.layers[:-5]:
  layer.trainable = True

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
#model.save_weights(model_save+'first_init.h5')

In [ ]:
model.load_weights(model_save+'9_init.h5')

In [ ]:
train_history_list = {'initial':[],'secondary':[]}
val_history_list = {'initial':[],'secondary':[]}


In [ ]:

for times in range(10):
    history = model.fit(x=train_generator,
                        epochs=5,
                        verbose=1,
                        )
    
    train_history_list['initial'].append(history)
    
    history = model.evaluate(x=val_generator,
                             verbose=1,
                             return_dict = True)
    
    val_history_list['initial'].append(history)

    model.save_weights(model_save+str(times)+'_init.h5')

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5
175/175 [==============================] - 211s 983ms/step - loss: 0.5708 - accuracy: 0.7833 - true_positive: 2219.6250 - true_negatives: 19336.6023
Epoch 2/5
175/175 [==============================] - 173s 985ms/step - loss: 0.4167 - accuracy: 0.8286 - true_positive: 2297.8580 - true_negatives: 19375.8977
Epoch 3/5
175/175 [==============================] - 172s 982ms/step - loss: 0.3962 - accuracy: 0.8423 - true_positive: 2326.7727 - true_negatives: 19405.0455
Epoch 4/5
175/175 [==============================] - 173s 988ms/step - loss: 0.4079 - accuracy: 0.8316 - true_positive: 2304.0966 - true_negatives: 19379.7955
Epoch 5/5
38/38 [==============================] - 29s 761ms/step - loss: 1.7297 - accuracy: 0.4158 - true_positive: 430.0000 - true_negatives: 7876.0000
Epoch 1/5
175/175 [==============================] - 174s 994ms/step - loss: 0.4043 - accuracy: 0.8336 - true_positive: 4600.0000 - true_negatives: 38358.0000
Epoch 2/5
175/175 [==============================] 

In [ ]:
#model.save_weights(model_save+str(times)+'_init.h5')
try:
  if not os.path.exists(base_dir+'losses'):
    os.mkdir(losses_dir)
except FileExistsError:
  print('Folder exists')


In [ ]:
train_history = []
for each in train_history_list['initial']:
  print(each.history)
  train_history.append(each.history)

val_history = []
for each in val_history_list['initial']:
  print(each)
  val_history.append(each)

{'loss': [0.4514133632183075, 0.421551376581192, 0.4063403308391571, 0.4092489182949066, 0.395191490650177], 'accuracy': [0.8160714507102966, 0.8257142901420593, 0.8305357098579407, 0.8310714364051819, 0.8392857313156128], 'true_positive': [4478.0, 4545.0, 4576.0, 4584.0, 4630.0], 'true_negatives': [38288.0, 38314.0, 38341.0, 38345.0, 38388.0]}
{'loss': [0.40432849526405334, 0.42640089988708496, 0.3937150835990906, 0.4089605212211609, 0.4024093747138977], 'accuracy': [0.8335714340209961, 0.8214285969734192, 0.8341071605682373, 0.8299999833106995, 0.833392858505249], 'true_positive': [4600.0, 4520.0, 4607.0, 4586.0, 4606.0], 'true_negatives': [38358.0, 38286.0, 38359.0, 38347.0, 38349.0]}
{'loss': [0.38712432980537415, 0.3970757722854614, 0.39098694920539856, 0.389972060918808, 0.4025404453277588], 'accuracy': [0.8424999713897705, 0.8382142782211304, 0.8364285826683044, 0.8335714340209961, 0.835357129573822], 'true_positive': [4652.0, 4615.0, 4618.0, 4611.0, 4608.0], 'true_negatives': [

In [ ]:
with open(losses_dir+'train_history.json','w') as f0:
  json.dump(train_history,f0)

with open(losses_dir+'val_history.json','w') as f0:
  json.dump(val_history,f0)

In [ ]:
for i,layer in enumerate(model.layers):
    print(i,layer)

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f3afb512cd0>
1 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3afad9f150>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3afa411f50>
3 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3afa12cc50>
4 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3afa130fd0>
5 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3afa13bcd0>
6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3afa140b50>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3afa0c6c50>
8 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3afa0cafd0>
9 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3afa0d2190>
10 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3afa0d6f50>
11 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3afa0ddf10>
12 <tensorflow.python.ke

In [ ]:
for layer in model.layers[12:]:
    layer.trainable = True

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
from_epoch = 0
num_of_epochs = 10

In [ ]:
#from_epoch = 20
#model.load_weights(model_save+'19_secondary.h5')

In [ ]:
for times in range(from_epoch,from_epoch+num_of_epochs+1,1):
    history = model.fit(x=train_generator,
                        epochs=5,
                        verbose=1,
                        )
    
    train_history_list['secondary'].append(history)
    
    history = model.evaluate(x=val_generator,
                             verbose=1,
                             return_dict = True)
    
    val_history_list['secondary'].append(history)

    model.save_weights(model_save+str(times)+'_secondary.h5')

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5
175/175 [==============================] - 142s 808ms/step - loss: 0.3452 - accuracy: 0.8546 - true_positive: 4733.0000 - true_negatives: 38448.0000
Epoch 2/5
175/175 [==============================] - 141s 807ms/step - loss: 0.3636 - accuracy: 0.8484 - true_positive: 4698.0000 - true_negatives: 38416.0000
Epoch 3/5
175/175 [==============================] - 142s 808ms/step - loss: 0.3544 - accuracy: 0.8511 - true_positive: 4711.0000 - true_negatives: 38433.0000
Epoch 4/5
175/175 [==============================] - 143s 819ms/step - loss: 0.3431 - accuracy: 0.8536 - true_positive: 4738.0000 - true_negatives: 38434.0000
Epoch 5/5
38/38 [==============================] - 20s 511ms/step - loss: 2.7414 - accuracy: 0.3525 - true_positive: 389.0000 - true_negatives: 7748.0000
Epoch 1/5
175/175 [==============================] - 143s 814ms/step - loss: 0.3306 - accuracy: 0.8641 - true_positive: 4802.0000 - true_negatives: 38489.0000
Epoch 2/5
175/175 [==============================] 

In [ ]:
train_history = []
for each in train_history_list['secondary']:
  print(each.history)
  train_history.append(each.history)

val_history = []
for each in val_history_list['secondary']:
  print(each)
  val_history.append(each)

In [ ]:
with open(losses_dir+'train_history_secondary3.json','w') as f0:
  json.dump(train_history,f0)

with open(losses_dir+'val_history_secondary3.json','w') as f0:
  json.dump(val_history,f0)

In [ ]:
for each in os.listdir(base_dir+'losses/'):
  print(each)